In [1]:
#import pyspark
#from pyspark import SparkContext
# from pyspark.sql.window import Window
# from pyspark.sql import functions as F
# from pyspark.sql import types as T

import json
import pandas as pd
import numpy as np

### Read Boarding Data

In [9]:
boarding_data = pd.read_json('/local/tarciso/masters/data/bus_trips/preliminary-exp-sample-data/ticketing-sample/doc1-2017051015.txt', lines=True)

In [10]:
boarding_data.head()

,CODLINHA,CODVEICULO,DATANASCIMENTO,DATAUTILIZACAO,NOMELINHA,NUMEROCARTAO,SEXO
0,000,00070,22/11/58,"09/05/17 11:05:57,000000",OPER S/LINHA,1353891,M
1,542,GA117,23/03/72,"09/05/17 17:25:14,000000",BAIRRO NOVO B,2357837,F
2,000,09053,23/03/72,"09/05/17 17:57:33,000000",OPER S/LINHA,2357837,F
3,000,09053,23/03/72,"09/05/17 17:57:28,000000",OPER S/LINHA,2357837,F
4,021,08046,26/01/72,"09/05/17 20:17:34,000000",INTERB II ANTI H,1937533,F


In [11]:
boardings_total = len(boarding_data)

In [12]:
boarding_data.dtypes

CODLINHA          object
CODVEICULO        object
DATANASCIMENTO    object
DATAUTILIZACAO    object
NOMELINHA         object
NUMEROCARTAO       int64
SEXO              object
dtype: object

#### Adding datetime column to ticketing dataframe 

In [13]:
boarding_data['DATAUTILIZACAO'].str.split(pat=',',expand=True)[0]

0         09/05/17 11:05:57
1         09/05/17 17:25:14
2         09/05/17 17:57:33
3         09/05/17 17:57:28
4         09/05/17 20:17:34
5         09/05/17 11:55:13
6         09/05/17 17:02:35
7         09/05/17 17:46:22
8         09/05/17 07:34:33
9         09/05/17 17:20:53
10        09/05/17 17:20:56
11        09/05/17 10:47:38
12        09/05/17 08:06:05
13        09/05/17 18:57:22
14        09/05/17 07:59:29
15        09/05/17 07:20:19
16        09/05/17 18:10:41
17        09/05/17 15:29:11
18        09/05/17 13:50:48
19        09/05/17 17:46:39
20        09/05/17 06:59:45
21        09/05/17 17:58:03
22        09/05/17 17:00:00
23        09/05/17 10:30:14
24        09/05/17 14:22:49
25        09/05/17 07:42:22
26        09/05/17 07:00:15
27        09/05/17 17:54:09
28        09/05/17 14:43:25
29        09/05/17 14:43:28
                ...        
320262    09/05/17 12:09:15
320263    09/05/17 17:32:09
320264    09/05/17 11:43:09
320265    09/05/17 17:55:38
320266    09/05/17 1

In [14]:
boarding_data['boarding_datetime'] = pd.to_datetime(boarding_data['DATAUTILIZACAO'].str.split(pat=',',expand=True)[0],format='%d/%m/%y %H:%M:%S')

#### Checking number of boardings on route 000

In [65]:
boardings_000 = len(boarding_data[boarding_data['CODLINHA'] == '000'])

In [66]:
print boardings_000, boardings_total, boardings_000/float(boardings_total)

134629 320292 0.420332071984


#### Looking at 000 boardings in more detail

In [67]:
boardings_000_df = boarding_data[boarding_data['CODLINHA'] == '000']

In [68]:
boardings_000_df.head(20)

,CODLINHA,CODVEICULO,DATANASCIMENTO,DATAUTILIZACAO,NOMELINHA,NUMEROCARTAO,SEXO,boarding_datetime
0,000,00070,22/11/58,"09/05/17 11:05:57,000000",OPER S/LINHA,1353891,M,2017-05-09 11:05:57
2,000,09053,23/03/72,"09/05/17 17:57:33,000000",OPER S/LINHA,2357837,F,2017-05-09 17:57:33
3,000,09053,23/03/72,"09/05/17 17:57:28,000000",OPER S/LINHA,2357837,F,2017-05-09 17:57:28
13,000,03009,28/08/84,"09/05/17 18:57:22,000000",OPER S/LINHA,3591720,F,2017-05-09 18:57:22
15,000,03031,29/12/63,"09/05/17 07:20:19,000000",OPER S/LINHA,1225547,M,2017-05-09 07:20:19
17,000,05040,17/01/79,"09/05/17 15:29:11,000000",OPER S/LINHA,2699543,F,2017-05-09 15:29:11
20,000,09005,14/09/76,"09/05/17 06:59:45,000000",OPER S/LINHA,2788103,F,2017-05-09 06:59:45
22,000,03037,01/06/89,"09/05/17 17:00:00,000000",OPER S/LINHA,2900987,F,2017-05-09 17:00:00
25,000,05056,02/07/63,"09/05/17 07:42:22,000000",OPER S/LINHA,1947358,M,2017-05-09 07:42:22
26,000,09029,22/02/65,"09/05/17 07:00:15,000000",OPER S/LINHA,3543165,M,2017-05-09 07:00:15


### Find boarding stations stop ids for line 000 boarding entries

#### Read Line 000 Codes translation table

In [69]:
line_000_terminals = pd.read_csv('/local/tarciso/masters/data/urbs/line-000-terminals-translation-table.csv', dtype = {'URBS_CODE': str})

In [70]:
line_000_terminals.head()

,LINE,NAME,URBS_CODE,STOP_ID,LAT,LON
0,000,TERMINAL BOQUEIRÃO,00001,14489,-25.516723,-49.230568
1,000,TERMINAL BOQUEIRÃO,00002,14489,-25.516723,-49.230568
2,000,TERMINAL BOQUEIRÃO,00003,14489,-25.516723,-49.230568
3,000,TERMINAL CARMO,00004,14485,-25.501311,-49.237825
4,000,TERMINAL CARMO,00005,14485,-25.501311,-49.237825


#### Match CODVEICULO to URBS_CODE to find line 000 boarding stations

In [71]:
matched_000_stations = boardings_000_df.merge(line_000_terminals, left_on='CODVEICULO', right_on='URBS_CODE', how='inner') \
                        .sort_values(by='DATAUTILIZACAO')

In [72]:
matched_000_stations.head(100)

,CODLINHA,CODVEICULO,DATANASCIMENTO,DATAUTILIZACAO,NOMELINHA,NUMEROCARTAO,SEXO,boarding_datetime,LINE,NAME,URBS_CODE,STOP_ID,LAT,LON
45746,000,00010,25/11/76,"09/05/17 04:47:52,000000",OPER S/LINHA,2337319,M,2017-05-09 04:47:52,000,TERMINAL SÍTIO CERCADO,00010,14492,-25.534235,-49.267368
46072,000,00010,22/09/66,"09/05/17 04:47:59,000000",OPER S/LINHA,3563770,M,2017-05-09 04:47:59,000,TERMINAL SÍTIO CERCADO,00010,14492,-25.534235,-49.267368
46180,000,00010,28/06/56,"09/05/17 04:48:16,000000",OPER S/LINHA,1799518,M,2017-05-09 04:48:16,000,TERMINAL SÍTIO CERCADO,00010,14492,-25.534235,-49.267368
96005,000,00012,29/03/77,"09/05/17 04:51:31,000000",OPER S/LINHA,3633298,M,2017-05-09 04:51:31,000,TERMINAL SÍTIO CERCADO,00012,14492,-25.534235,-49.267368
81008,000,09003,19/11/52,"09/05/17 04:52:03,000000",OPER S/LINHA,1503781,F,2017-05-09 04:52:03,000,TERMINAL FAZENDINHA,09003,14512,-25.477397,-49.326660
96002,000,00012,10/05/62,"09/05/17 04:52:04,000000",OPER S/LINHA,3316674,F,2017-05-09 04:52:04,000,TERMINAL SÍTIO CERCADO,00012,14492,-25.534235,-49.267368
96417,000,00012,12/01/78,"09/05/17 04:52:22,000000",OPER S/LINHA,3702630,F,2017-05-09 04:52:22,000,TERMINAL SÍTIO CERCADO,00012,14492,-25.534235,-49.267368
30401,000,03001,27/11/58,"09/05/17 04:53:01,000000",OPER S/LINHA,2441463,M,2017-05-09 04:53:01,000,TERMINAL PINHEIRINHO,03001,14494,-25.512983,-49.294505
96358,000,00012,17/07/68,"09/05/17 04:53:15,000000",OPER S/LINHA,3325352,M,2017-05-09 04:53:15,000,TERMINAL SÍTIO CERCADO,00012,14492,-25.534235,-49.267368
95563,000,00012,16/07/94,"09/05/17 04:53:58,000000",OPER S/LINHA,3727031,M,2017-05-09 04:53:58,000,TERMINAL SÍTIO CERCADO,00012,14492,-25.534235,-49.267368


In [73]:
num_matched_stations = len(matched_000_stations)

In [74]:
print num_matched_stations, boardings_000, num_matched_stations/float(boardings_000)

122864 134629 0.912611695846


### Find Candidate bus routes for each boarding event

#### Read GPS data for the same day

In [16]:
#gps_data = pd.read_csv('/local/tarciso/masters/data/bus_trips/buste-v3a/may_jun_jul/single-csvs/2017_05_09_veiculos.csv', dtype = {'route': str}, na_values='-')
gps_data = pd.read_csv('/local/tarciso/masters/data/bus_trips/test/buste-maio/2017_05_10_veiculos.csv', dtype = {'route': str}, na_values='-')

/local/tarciso/programs/anaconda2/lib/python2.7/site-packages/IPython/core/interactiveshell.py:2718: DtypeWarning: Columns (7,12,15,16,17,19) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [17]:
gps_data.head()

,route,tripNum,shapeId,shapeSequence,shapeLat,shapeLon,distanceTraveledShape,busCode,gpsPointId,gpsLat,gpsLon,distanceToShapePoint,timestamp,stopPointId,problem,birthdate,cardTimestamp,lineName,cardNum,gender
0,500,2.0,4130,6464167,-25.433790,-49.270262,10543.422,GE718,NaN,-25.433916,-49.270425,21.556105,06:58:25,27560,NO_PROBLEM,NaN,NaN,NaN,NaN,NaN
1,500,4.0,4130,6464041,-25.459174,-49.258160,7268.131,GE718,NaN,-25.459181,-49.258193,3.460972,07:07:26,25471,TRIP_PROBLEM,NaN,NaN,NaN,NaN,NaN
2,500,5.0,4130,6463945,-25.481825,-49.246977,4513.260,GE718,NaN,NaN,NaN,NaN,07:57:57,27551,BETWEEN,NaN,NaN,NaN,NaN,NaN
3,500,3.0,4130,6463945,-25.481825,-49.246977,4513.260,GE718,NaN,-25.481915,-49.247050,12.410537,08:00:09,27551,TRIP_PROBLEM,NaN,NaN,NaN,NaN,NaN
4,500,1.0,4130,6463793,-25.516848,-49.230057,0.000,GE718,NaN,-25.516915,-49.229840,23.004353,08:10:07,27537,TRIP_PROBLEM,NaN,NaN,NaN,NaN,NaN


In [18]:
gps_data.dtypes

route                     object
tripNum                  float64
shapeId                    int64
shapeSequence              int64
shapeLat                 float64
shapeLon                 float64
distanceTraveledShape    float64
busCode                   object
gpsPointId               float64
gpsLat                   float64
gpsLon                   float64
distanceToShapePoint     float64
timestamp                 object
stopPointId                int64
problem                   object
birthdate                 object
cardTimestamp             object
lineName                  object
cardNum                  float64
gender                    object
dtype: object

In [19]:
len(gps_data)

1298680

#### Analyzing Boarding and GPS Data

In [20]:
boarding_data[['NUMEROCARTAO','DATAUTILIZACAO','CODLINHA','CODVEICULO']] \
    .sort_values(['NUMEROCARTAO','DATAUTILIZACAO']).head(100)

,NUMEROCARTAO,DATAUTILIZACAO,CODLINHA,CODVEICULO
243380,228696,"09/05/17 07:01:00,000000",000,08024
243381,228696,"09/05/17 13:55:35,000000",000,05312
61748,229948,"09/05/17 06:29:09,000000",654,HA017
199681,233641,"09/05/17 06:39:21,000000",000,01026
199680,233641,"09/05/17 12:08:17,000000",021,04020
199752,257342,"09/05/17 07:48:58,000000",511,EA172
199751,257342,"09/05/17 17:24:51,000000",000,01021
14035,272904,"09/05/17 17:24:52,000000",000,03047
199773,300327,"09/05/17 06:56:45,000000",OPC,HA240
199772,300327,"09/05/17 16:52:22,000000",000,03057


In [21]:
gps_data[['cardNum','cardTimestamp','timestamp','route','busCode','stopPointId']] \
    .sort_values(['cardNum','cardTimestamp']) \
    .head(100)

,cardNum,cardTimestamp,timestamp,route,busCode,stopPointId
1140069,229948.0,06:29:09,06:20:40,654,HA017,32429
1140684,229948.0,06:29:09,06:28:51,654,HA017,35349
1140935,229948.0,06:29:09,12:55:17,654,HA017,35927
1141556,229948.0,06:29:09,06:17:11,654,HA017,36102
609704,257342.0,07:48:58,07:38:32,511,EA172,40444
610234,257342.0,07:48:58,07:10:28,511,EA172,28437
610545,257342.0,07:48:58,06:06:03,511,EA172,29827
610876,257342.0,07:48:58,07:48:14,511,EA172,31195
866834,300327.0,06:56:45,06:47:36,654,HA240,36086
866918,300327.0,06:56:45,06:55:54,654,HA240,36094


#### Add date and datetime to gps dataframe

In [79]:
gps_data['date'] = '2017-05-09'
gps_data['gps_datetime'] = pd.to_datetime(gps_data['date'] + ' ' + gps_data['timestamp'],format='%Y-%m-%d %H:%M:%S')
gps_data['date'] = pd.to_datetime(gps_data['date'], format='%Y-%m-%d') 

In [81]:
gps_data[['date','gps_datetime']]

,date,gps_datetime
0,2017-05-09,2017-05-09 06:53:44
1,2017-05-09,2017-05-09 06:58:25
2,2017-05-09,2017-05-09 07:22:26
3,2017-05-09,2017-05-09 07:26:47
4,2017-05-09,2017-05-09 07:28:04
5,2017-05-09,2017-05-09 07:55:16
6,2017-05-09,2017-05-09 07:57:57
7,2017-05-09,2017-05-09 08:00:09
8,2017-05-09,2017-05-09 06:42:45
9,2017-05-09,2017-05-09 08:10:07


### Match BUSTE GPS entries with their respective parent stop codes

#### Read GTFS stops file

In [82]:
stops_df = pd.read_csv('/local/tarciso/data/gtfs/curitiba/stops.txt')

In [83]:
stops_df.head()

,stop_id,stop_code,stop_name,stop_desc,stop_lat,stop_lon,zone_id,stop_url,location_type,parent_station,stop_timezone,wheelchair_boarding
0,70,104505,Terminal Campina do Siqueira - 303 - Centenári...,Terminal Campina do Siqueira - Campo Comprido,-25.435724,-49.306998,NaN,NaN,0,14506.0,NaN,NaN
1,270,104905,Terminal Carmo - 030 - Interbairros III,Terminal Carmo - 030 - Interbairros III (Senti...,-25.501341,-49.237597,NaN,NaN,0,14485.0,NaN,NaN
2,276,105606,Terminal Oficinas - 030 - Interbairros III,Terminal Oficinas - 030 - Interbairros III (Se...,-25.451550,-49.214917,NaN,NaN,0,14481.0,NaN,NaN
3,299,105603,Terminal Oficinas - 030 - Interbairros III,Terminal Oficinas - 030 - Interbairros III (Se...,-25.451665,-49.215086,NaN,NaN,0,14481.0,NaN,NaN
4,308,104907,Terminal Carmo - 030 - Interbairros III,Terminal Carmo - 030 - Interbairros III (Senti...,-25.501311,-49.237825,NaN,NaN,0,14485.0,NaN,NaN


In [84]:
stops_parent_stations = stops_df[['stop_id','parent_station']]

#### Add parent station ID to GPS Boardings data

In [85]:
gps_stops_with_parent_stations = gps_data.merge(stops_parent_stations, left_on='stopPointId', right_on='stop_id', how='inner')

In [86]:
gps_stops_with_parent_stations.head()

,route,tripNum,shapeId,shapeSequence,shapeLat,shapeLon,distanceTraveledShape,busCode,gpsPointId,gpsLat,...,problem,birthdate,cardTimestamp,lineName,cardNum,gender,date,gps_datetime,stop_id,parent_station
0,500,2.0,4130,6464140,-25.438837,-49.268129,9735.966,GE718,NaN,-25.438738,...,NO_PROBLEM,NaN,NaN,NaN,NaN,NaN,2017-05-09,2017-05-09 06:53:44,25515,NaN
1,500,4.0,4130,6464140,-25.438837,-49.268129,9735.966,GE718,NaN,-25.438743,...,TRIP_PROBLEM,NaN,NaN,NaN,NaN,NaN,2017-05-09,2017-05-09 18:26:05,25515,NaN
2,500,3.0,4130,6464140,-25.438837,-49.268129,9735.966,GE718,NaN,NaN,...,BETWEEN,NaN,NaN,NaN,NaN,NaN,2017-05-09,2017-05-09 07:40:50,25515,NaN
3,500,5.0,4130,6464140,-25.438837,-49.268129,9735.966,GE718,NaN,NaN,...,BETWEEN,NaN,NaN,NaN,NaN,NaN,2017-05-09,2017-05-09 08:31:53,25515,NaN
4,550,1.0,2000,4985410,-25.438978,-49.268149,10308.128,NaN,NaN,NaN,...,BETWEEN,NaN,NaN,NaN,NaN,NaN,2017-05-09,NaT,25515,NaN


#### Finding buses for the following boarding entry:

In [87]:
matched_000_stations[:1]

,CODLINHA,CODVEICULO,DATANASCIMENTO,DATAUTILIZACAO,NOMELINHA,NUMEROCARTAO,SEXO,boarding_datetime,LINE,NAME,URBS_CODE,STOP_ID,LAT,LON
45746,000,00010,25/11/76,"09/05/17 04:47:52,000000",OPER S/LINHA,2337319,M,2017-05-09 04:47:52,000,TERMINAL SÍTIO CERCADO,00010,14492,-25.534235,-49.267368


In [88]:
type(gps_stops_with_parent_stations['stopPointId'][0])

numpy.int64

#### Select candidate bus/trip records by filtering the ones which did not go throught the boarding station

In [128]:
candidate_bus_trips = gps_stops_with_parent_stations.query('parent_station==14492') \
                        .drop_duplicates(subset=['route','busCode','tripNum']) \
                        .sort_values(by='gps_datetime')

In [129]:
candidate_bus_trips

,route,tripNum,shapeId,shapeSequence,shapeLat,shapeLon,distanceTraveledShape,busCode,gpsPointId,gpsLat,...,problem,birthdate,cardTimestamp,lineName,cardNum,gender,date,gps_datetime,stop_id,parent_station
1205282,547,21.0,1994,6165177,-25.534173,-49.267628,0.000,GA146,NaN,-25.534175,...,NO_PROBLEM,NaN,NaN,NaN,NaN,NaN,2017-05-09,2017-05-09 04:54:15,30013,14492.0
883278,542,13.0,1988,6404092,-25.534141,-49.267486,7134.743,GA124,NaN,-25.534153,...,NO_PROBLEM,09/04/98,05:28:07,BAIRRO NOVO B,3243753.0,F,2017-05-09,2017-05-09 04:58:58,30012,14492.0
1164869,548,20.0,3581,6568518,-25.534214,-49.267795,0.000,GA205,NaN,-25.534248,...,NO_PROBLEM,NaN,NaN,NaN,NaN,NaN,2017-05-09,2017-05-09 05:01:29,4661,14492.0
883276,542,14.0,1988,6404092,-25.534141,-49.267486,7134.743,GA124,NaN,-25.534175,...,NO_PROBLEM,NaN,NaN,NaN,NaN,NaN,2017-05-09,2017-05-09 05:03:59,30012,14492.0
1205132,547,22.0,1994,6165177,-25.534173,-49.267628,0.000,GA146,NaN,-25.534195,...,NO_PROBLEM,12/03/85,05:08:22,BAIRRO NOVO C,3090669.0,F,2017-05-09,2017-05-09 05:07:50,30013,14492.0
545583,502,1.0,3241,6256532,-25.534277,-49.267408,5176.189,GR028,NaN,-25.534286,...,NO_PROBLEM,NaN,NaN,NaN,NaN,NaN,2017-05-09,2017-05-09 05:15:17,27079,14492.0
1164826,548,3.0,3580,5955066,-25.534213,-49.267794,8750.881,GA204,NaN,-25.534235,...,TRIP_PROBLEM,NaN,NaN,NaN,NaN,NaN,2017-05-09,2017-05-09 05:15:31,4661,14492.0
66838,540,1.0,1985,6328328,-25.534398,-49.267898,4568.548,GR409,NaN,NaN,...,BETWEEN,01/05/80,05:19:36,OP. CONTIGENCIA,2394563.0,F,2017-05-09,2017-05-09 05:18:56,5346,14492.0
883749,542,7.0,1988,6404092,-25.534141,-49.267486,7134.743,GA143,NaN,-25.534195,...,NO_PROBLEM,NaN,NaN,NaN,NaN,NaN,2017-05-09,2017-05-09 05:22:33,30012,14492.0
1205133,547,30.0,1994,6165177,-25.534173,-49.267628,0.000,GA146,NaN,-25.534210,...,NO_PROBLEM,NaN,NaN,NaN,NaN,NaN,2017-05-09,2017-05-09 05:24:11,30013,14492.0


In [130]:
candidate_bus_trips['diff_boarding_arrival'] = np.abs(candidate_bus_trips['gps_datetime'] - pd.to_datetime('2017-05-09 04:47:52', format='%Y-%m-%d %H:%M:%S'))
candidate_bus_trips = candidate_bus_trips[candidate_bus_trips['diff_boarding_arrival'] < pd.Timedelta('1 hour')]

In [142]:
candidate_bus_trips.head(10)

,route,tripNum,shapeId,shapeSequence,shapeLat,shapeLon,distanceTraveledShape,busCode,gpsPointId,gpsLat,...,birthdate,cardTimestamp,lineName,cardNum,gender,date,gps_datetime,stop_id,parent_station,diff_boarding_arrival
1205282,547,21.0,1994,6165177,-25.534173,-49.267628,0.000,GA146,NaN,-25.534175,...,NaN,NaN,NaN,NaN,NaN,2017-05-09,2017-05-09 04:54:15,30013,14492.0,00:06:23
883278,542,13.0,1988,6404092,-25.534141,-49.267486,7134.743,GA124,NaN,-25.534153,...,09/04/98,05:28:07,BAIRRO NOVO B,3243753.0,F,2017-05-09,2017-05-09 04:58:58,30012,14492.0,00:11:06
1164869,548,20.0,3581,6568518,-25.534214,-49.267795,0.000,GA205,NaN,-25.534248,...,NaN,NaN,NaN,NaN,NaN,2017-05-09,2017-05-09 05:01:29,4661,14492.0,00:13:37
883276,542,14.0,1988,6404092,-25.534141,-49.267486,7134.743,GA124,NaN,-25.534175,...,NaN,NaN,NaN,NaN,NaN,2017-05-09,2017-05-09 05:03:59,30012,14492.0,00:16:07
1205132,547,22.0,1994,6165177,-25.534173,-49.267628,0.000,GA146,NaN,-25.534195,...,12/03/85,05:08:22,BAIRRO NOVO C,3090669.0,F,2017-05-09,2017-05-09 05:07:50,30013,14492.0,00:19:58
545583,502,1.0,3241,6256532,-25.534277,-49.267408,5176.189,GR028,NaN,-25.534286,...,NaN,NaN,NaN,NaN,NaN,2017-05-09,2017-05-09 05:15:17,27079,14492.0,00:27:25
1164826,548,3.0,3580,5955066,-25.534213,-49.267794,8750.881,GA204,NaN,-25.534235,...,NaN,NaN,NaN,NaN,NaN,2017-05-09,2017-05-09 05:15:31,4661,14492.0,00:27:39
66838,540,1.0,1985,6328328,-25.534398,-49.267898,4568.548,GR409,NaN,NaN,...,01/05/80,05:19:36,OP. CONTIGENCIA,2394563.0,F,2017-05-09,2017-05-09 05:18:56,5346,14492.0,00:31:04
883749,542,7.0,1988,6404092,-25.534141,-49.267486,7134.743,GA143,NaN,-25.534195,...,NaN,NaN,NaN,NaN,NaN,2017-05-09,2017-05-09 05:22:33,30012,14492.0,00:34:41
1205133,547,30.0,1994,6165177,-25.534173,-49.267628,0.000,GA146,NaN,-25.534210,...,NaN,NaN,NaN,NaN,NaN,2017-05-09,2017-05-09 05:24:11,30013,14492.0,00:36:19


In [149]:
def get_arrival_time_at_next_stop_after_station(gps_df,route,vehicle,trip,station_timestamp):
        return gps_df[((gps_df['route'] == route) & (gps_df['busCode'] == vehicle)) \
                      & ((gps_df['tripNum'] == trip) & (gps_df['gps_datetime'] > station_timestamp))] \
                .drop_duplicates(subset=['route','busCode','tripNum','stopPointId']) \
                .sort_values('gps_datetime')
    

In [150]:
get_arrival_time_at_next_stop_after_station(gps_stops_with_parent_stations,'542','GA124',13,pd.to_datetime('2017-05-09 04:47:52', format='%Y-%m-%d %H:%M:%S'))

,route,tripNum,shapeId,shapeSequence,shapeLat,shapeLon,distanceTraveledShape,busCode,gpsPointId,gpsLat,...,problem,birthdate,cardTimestamp,lineName,cardNum,gender,date,gps_datetime,stop_id,parent_station
739654,542,13.0,1988,6403957,-25.551388,-49.266807,3278.756,GA124,NaN,-25.551363,...,NO_PROBLEM,02/02/58,04:48:50,BAIRRO NOVO B,3696327.0,F,2017-05-09,2017-05-09 04:48:22,31255,NaN
739292,542,13.0,1988,6403964,-25.550431,-49.265282,3509.356,GA124,NaN,-25.550511,...,NO_PROBLEM,19/02/80,04:49:15,BAIRRO NOVO B,785212.0,F,2017-05-09,2017-05-09 04:48:59,31256,NaN
738466,542,13.0,1988,6403971,-25.551567,-49.262789,3814.244,GA124,NaN,-25.551588,...,NO_PROBLEM,15/08/63,04:50:49,BAIRRO NOVO B,931864.0,F,2017-05-09,2017-05-09 04:49:55,31257,NaN
738127,542,13.0,1988,6403979,-25.550383,-49.261133,4070.023,GA124,NaN,-25.550470,...,NO_PROBLEM,NaN,NaN,NaN,NaN,NaN,2017-05-09,2017-05-09 04:51:05,31258,NaN
737735,542,13.0,1988,6403984,-25.549763,-49.259976,4240.869,GA124,NaN,-25.549690,...,NO_PROBLEM,12/04/94,04:51:57,BAIRRO NOVO B,3824438.0,F,2017-05-09,2017-05-09 04:51:40,31259,NaN
738790,542,13.0,1988,6403992,-25.547981,-49.259658,4518.448,GA124,NaN,-25.547955,...,NO_PROBLEM,17/04/62,04:53:27,BAIRRO NOVO B,1119175.0,F,2017-05-09,2017-05-09 04:52:48,31260,NaN
739922,542,13.0,1988,6404006,-25.545885,-49.263491,4968.473,GA124,NaN,-25.545811,...,NO_PROBLEM,NaN,NaN,NaN,NaN,NaN,2017-05-09,2017-05-09 04:53:41,31261,NaN
740736,542,13.0,1988,6404013,-25.545020,-49.265058,5152.768,GA124,NaN,-25.544915,...,NO_PROBLEM,05/01/84,04:55:11,BAIRRO NOVO B,1118881.0,F,2017-05-09,2017-05-09 04:54:19,31262,NaN
742066,542,13.0,1988,6404027,-25.543177,-49.268451,5550.298,GA124,NaN,-25.543125,...,NO_PROBLEM,07/07/78,04:55:22,BAIRRO NOVO B,3788948.0,M,2017-05-09,2017-05-09 04:55:19,31250,NaN
743443,542,13.0,1988,6404044,-25.541586,-49.270588,5838.633,GA124,NaN,NaN,...,BETWEEN,13/08/59,04:56:09,BAIRRO NOVO B,1342841.0,F,2017-05-09,2017-05-09 04:56:00,31263,NaN


#### Concatenate line 000 boardings with GPS boardings

In [151]:
matched_000_stations.head()

,CODLINHA,CODVEICULO,DATANASCIMENTO,DATAUTILIZACAO,NOMELINHA,NUMEROCARTAO,SEXO,boarding_datetime,LINE,NAME,URBS_CODE,STOP_ID,LAT,LON
45746,000,00010,25/11/76,"09/05/17 04:47:52,000000",OPER S/LINHA,2337319,M,2017-05-09 04:47:52,000,TERMINAL SÍTIO CERCADO,00010,14492,-25.534235,-49.267368
46072,000,00010,22/09/66,"09/05/17 04:47:59,000000",OPER S/LINHA,3563770,M,2017-05-09 04:47:59,000,TERMINAL SÍTIO CERCADO,00010,14492,-25.534235,-49.267368
46180,000,00010,28/06/56,"09/05/17 04:48:16,000000",OPER S/LINHA,1799518,M,2017-05-09 04:48:16,000,TERMINAL SÍTIO CERCADO,00010,14492,-25.534235,-49.267368
96005,000,00012,29/03/77,"09/05/17 04:51:31,000000",OPER S/LINHA,3633298,M,2017-05-09 04:51:31,000,TERMINAL SÍTIO CERCADO,00012,14492,-25.534235,-49.267368
81008,000,09003,19/11/52,"09/05/17 04:52:03,000000",OPER S/LINHA,1503781,F,2017-05-09 04:52:03,000,TERMINAL FAZENDINHA,09003,14512,-25.477397,-49.326660


In [166]:
line_000_boardings = matched_000_stations.drop(['LINE','NAME','URBS_CODE','LAT','LON','boarding_datetime'], axis=1) \
                        .rename(index=str, columns={'CODLINHA': 'route', 'CODVEICULO': 'busCode', 
                                                    'DATANASCIMENTO':'birthdate','DATAUTILIZACAO':'cardTimestamp',
                                                    'NOMELINHA':'lineName','NUMEROCARTAO':'cardNum',
                                                     'SEXO':'gender','STOP_ID':'stopPointId'})

In [165]:
line_000_boardings.head()

,route,busCode,birthdate,cardTimestamp,lineName,cardNum,gender,boarding_datetime,stopPointId
45746,000,00010,25/11/76,"09/05/17 04:47:52,000000",OPER S/LINHA,2337319,M,2017-05-09 04:47:52,14492
46072,000,00010,22/09/66,"09/05/17 04:47:59,000000",OPER S/LINHA,3563770,M,2017-05-09 04:47:59,14492
46180,000,00010,28/06/56,"09/05/17 04:48:16,000000",OPER S/LINHA,1799518,M,2017-05-09 04:48:16,14492
96005,000,00012,29/03/77,"09/05/17 04:51:31,000000",OPER S/LINHA,3633298,M,2017-05-09 04:51:31,14492
81008,000,09003,19/11/52,"09/05/17 04:52:03,000000",OPER S/LINHA,1503781,F,2017-05-09 04:52:03,14512


In [163]:
gps_stops_with_parent_stations.columns

Index([u'route', u'tripNum', u'shapeId', u'shapeSequence', u'shapeLat',
       u'shapeLon', u'distanceTraveledShape', u'busCode', u'gpsPointId',
       u'gpsLat', u'gpsLon', u'distanceToShapePoint', u'timestamp',
       u'stopPointId', u'problem', u'birthdate', u'cardTimestamp', u'lineName',
       u'cardNum', u'gender', u'date', u'gps_datetime', u'stop_id',
       u'parent_station'],
      dtype='object')

In [177]:
gps_boardings_with_000 = pd.concat([gps_stops_with_parent_stations,line_000_boardings]) \
                            .drop('stop_id', axis=1)

In [180]:
gps_boardings_with_000 = gps_boardings_with_000[['route','busCode','tripNum','stopPointId','parent_station','cardNum','cardTimestamp','gps_datetime',
                                      'shapeId','shapeSequence','shapeLat','shapeLon','distanceTraveledShape','gpsPointId',
                                      'gpsLat','gpsLon','distanceToShapePoint','timestamp','problem','lineName','birthdate',
                                      'lineName','gender','date']] \
                            .sort_values(['cardNum','cardTimestamp'])

In [181]:
gps_boardings_with_000

,route,busCode,tripNum,stopPointId,parent_station,cardNum,cardTimestamp,gps_datetime,shapeId,shapeSequence,...,distanceToShapePoint,timestamp,problem,lineName,lineName,birthdate,lineName,lineName,gender,date
109722,000,08024,NaN,41752,NaN,228696.0,"09/05/17 07:01:00,000000",NaT,NaN,NaN,...,NaN,NaN,NaN,OPER S/LINHA,OPER S/LINHA,27/04/94,OPER S/LINHA,OPER S/LINHA,F,NaT
756925,654,HA017,39.0,35927,NaN,229948.0,06:29:09,2017-05-09 12:55:17,2106.0,6040805.0,...,6.075716,12:55:17,NO_PROBLEM,CAMPO ALEGRE,CAMPO ALEGRE,28/04/95,CAMPO ALEGRE,CAMPO ALEGRE,F,2017-05-09
810134,654,HA017,1.0,35349,NaN,229948.0,06:29:09,2017-05-09 06:28:51,2106.0,6040781.0,...,20.905151,06:28:51,TRIP_PROBLEM,CAMPO ALEGRE,CAMPO ALEGRE,28/04/95,CAMPO ALEGRE,CAMPO ALEGRE,F,2017-05-09
1141215,654,HA017,26.0,28610,14503.0,229948.0,06:29:09,2017-05-09 05:34:54,2106.0,6040943.0,...,3.586574,05:34:54,NO_PROBLEM,CAMPO ALEGRE,CAMPO ALEGRE,28/04/95,CAMPO ALEGRE,CAMPO ALEGRE,F,2017-05-09
1142836,654,HA017,18.0,27485,14497.0,229948.0,06:29:09,2017-05-09 12:02:21,2106.0,6040759.0,...,11.171079,12:02:21,TRIP_PROBLEM,CAMPO ALEGRE,CAMPO ALEGRE,28/04/95,CAMPO ALEGRE,CAMPO ALEGRE,F,2017-05-09
1145283,654,HA017,11.0,36093,NaN,229948.0,06:29:09,2017-05-09 06:07:15,2106.0,6040864.0,...,NaN,06:07:15,BETWEEN,CAMPO ALEGRE,CAMPO ALEGRE,28/04/95,CAMPO ALEGRE,CAMPO ALEGRE,F,2017-05-09
21774,000,01026,NaN,41863,NaN,233641.0,"09/05/17 06:39:21,000000",NaT,NaN,NaN,...,NaN,NaN,NaN,OPER S/LINHA,OPER S/LINHA,20/09/96,OPER S/LINHA,OPER S/LINHA,F,NaT
1196632,511,EA172,3.0,27747,14487.0,257342.0,07:48:58,2017-05-09 07:24:56,2746.0,5846226.0,...,200.947850,07:24:56,TRIP_PROBLEM,SÃO FRANCISCO,SÃO FRANCISCO,24/01/71,SÃO FRANCISCO,SÃO FRANCISCO,F,2017-05-09
1197822,511,EA172,21.0,28431,NaN,257342.0,07:48:58,2017-05-09 07:36:18,2746.0,5846080.0,...,6.291704,07:36:18,NO_PROBLEM,SÃO FRANCISCO,SÃO FRANCISCO,24/01/71,SÃO FRANCISCO,SÃO FRANCISCO,F,2017-05-09
1234923,511,EA172,51.0,40533,NaN,257342.0,07:48:58,2017-05-09 07:19:54,2746.0,5846198.0,...,3.972438,07:19:54,NO_PROBLEM,SÃO FRANCISCO,SÃO FRANCISCO,24/01/71,SÃO FRANCISCO,SÃO FRANCISCO,F,2017-05-09


In [90]:
gps_stops[gps_stops['stopPointId'] == 14487]

,stopPointId


In [86]:
gps_stops.merge(line_000_terminals, left_on='stopPointId', right_on='STOP_ID', how='inner')

,stopPointId,LINE,NAME,URBS_CODE,STOP_ID,LAT,LON
